In [1]:
import copy

from Game import * 
import params

In [2]:
class Node:
    def __init__(self):
        self.edges = []
        

In [3]:
class Edge:
    def __init__(self, x, y, node = None, p = 0.0):
        self.N = 0
        self.W = 0.0
        self.Q = 0.0
        self.P = p
        self.act_x = x
        self.act_y = y
        
        self.result_node = node       
        

In [4]:
class MCTS:
    def __init__(self, game, evaluator, root_node = None):
        self.root_node = root_node
        if self.root_node is None:
            self.root_node = Node()
        self.game = game
        self.evaluator = evaluator
            
            
    def select_move(self):
        for i in range(params.MCTS_NR):
            g = copy.deepcopy(self.game)
            self.reach_leaf_node_and_update_edges_and_extend(g)
        if params.MCTS_COMP:
            maxN = -1
            arr = []
            for edge in self.root_node.edges:
                if edge.N > maxN:
                    maxN = edge.N
                    arr=[]
                    arr.append(edge)
                elif edge.N == maxN:
                    arr.append(edge)                    
            edge = np.random.choice(arr)            
        else:
            prob=np.zeros( (len(self.root_node.edges)) )
            for i in range(len(self.root_node.edges)):
                prob[i] = float(self.root_node.edges[i].N)**(1.0/params.MCTS_TAU)
            prob /= sum(prob)
            index = np.random.choice( len(self.root_node.edges), p=prob)
            edge = self.root_node.edges[index]

        g = copy.deepcopy(self.game)
        g.move(edge.act_x, edge.act_y)
        return edge.act_x, edge.act_y, edge.result_node, g

    def reach_leaf_node_and_update_edges_and_extend(self,g):
        curr = self.root_node
        route = []
        while len(curr.edges) != 0:
            prob=np.zeros( (len(curr.edges)) )
            for i in range(len(curr.edges)):
                Q = curr.edges[i].Q
                U = params.MCTS_U_COEFF * curr.edges[i].P / (1.0 + curr.edges[i].N)
                prob[i] = Q + U
            prob /= sum(prob)
            index = np.random.choice( len(curr.edges), p=prob)
            edge = curr.edges[index]
            route.append(edge)
            curr = edge.result_node
            g.move(edge.act_x, edge.act_y)
        
        p = None
        if g.winner == 1:   # X
            v = 0.0
        elif g.winner == 2: # O
            v = 1.0
        elif g.winner == 3: # Draw
            v = 0.5
        else: # Not terminating state
            p,v = self.evaluator(g)
            
        # Extending the current node
        if p is not None:
            for x in range(15):
                for y in  range(15):
                    if g.grid[x,y] == 0:
                        n = Node()
                        e = Edge(x,y,n,p[x,y])
                        curr.edges.append(e)
        
        # Update edges on the route
        for i in range( len(route)-1, -1, -1):
            route[i].N += 1
            route[i].W += v
            route[i].Q = route[i].W / route[i].N
            v = 1 - v 

        return g, curr    
            

In [5]:
#uni_probs = np.ones( (15,15) )
#uni_probs /= 225.0


uni_probs = np.ones( (15,15) )
for x in range(15):
    for y in range(15):
        uni_probs[x,y] = 1.0 / (abs(7.0-x) + abs(7.0-y) + 1.0)**3
uni_probs /= sum(sum(uni_probs))


In [6]:
def evaluator(g):
    return uni_probs, 0.5

g = Game()
nr=0
while (g.winner==0):
    if nr%2==0:
        x = np.random.randint(15)
        y = np.random.randint(15)
        while g.grid[x,y]!=0:
            x = np.random.randint(15)
            y = np.random.randint(15)
        g.move(x,y)
        g.print()
    else:
        mcts = MCTS(g, evaluator)
        x,y,_,g = mcts.select_move()
        g.print() 
    nr+=1
print(nr, g.winner)


-----------------
|               |
|               |
|               |
|               |
|           X   |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|           X   |
|               |
|               |
|       O       |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|           X   |
|     X         |
|               |
|       O       |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|       O 

In [7]:
def evaluator(g):
    return uni_probs, 0.5

g = Game()
n = None
nr = 0
while (g.winner==0):
    mcts = MCTS(g, evaluator, n)
    x,y,n,g = mcts.select_move()
    g.print() 
    nr+=1
print(nr,g.winner)


-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|               |
|         X     |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|               |
|               |
|       O       |
|         X     |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|               |
|               |
|        X      |
|       O       |
|         X     |
|               |
|               |
|               |
|               |
|               |
|               |
|               |
-----------------
-----------------
|               |
|               |
|               |
|         